# Projet DATA732

On ajoute les librairies utilisées

In [2]:
import json
import os
import pprint 
import plotly.express as px

## Analyse du nombre d'articles

On crée le corpus. Il est composé de 4 .json pour les 4 sites 

In [30]:
corpusFiles = ["topaz-data732--all--www.egaliteetreconciliation.fr--20190101--20211231.json", 
"topaz-data732--all--fr.sputniknews.africa--20190101--20211231.json", 
"topaz-data732--all--french.presstv.ir--20190101--20211231.json",
"topaz-data732--all--www.fdesouche.com--20190101--20211231.json"]

On crée une fonction nous permettant de connaître le nombre d'articles par mois. On sépare les données selon le site

In [32]:
def getNumberArticlePerMonth(filesNames):
    numberArticlePerMonth = []
    for file in filesNames:
        # open and load file
        f = open(file, 'r', encoding='utf-8')
        data = json.loads(f.read())
        f.close()
        siteName = f.name.split(".")[1]
        years = data["metadata-all"]["fr"]["month"].keys()
        for year in years:
            for month in data["metadata-all"]["fr"]["month"][year].keys():
                num = data["metadata-all"]["fr"]["month"][year][month]["num"]
                numberArticlePerMonth.append({"yearMonth" : (month + "-" + year), "count" : num, "site": siteName})

    #BubbleSort
    good = False

    while(not good):
        good = True
        for i in range(0, len(numberArticlePerMonth) - 1):
            date1 = numberArticlePerMonth[i]["yearMonth"]
            date2 = numberArticlePerMonth[i+1]["yearMonth"]
            (month1, annee1) = date1.split("-")
            (month2, annee2) = date2.split("-")
            if(int(annee1) > int(annee2)):
                numberArticlePerMonth[i], numberArticlePerMonth[i+1] = numberArticlePerMonth[i+1], numberArticlePerMonth[i]
                good = False
            if(annee1 == annee2 and int(month2) < int(month1)):
                numberArticlePerMonth[i], numberArticlePerMonth[i+1] = numberArticlePerMonth[i+1], numberArticlePerMonth[i]
                good = False

    return numberArticlePerMonth

In [33]:
number_article_per_month = getNumberArticlePerMonth(corpusFiles)

On crée un bar chart du nombre d'articles par mois et par site.

In [34]:
fig = px.bar(number_article_per_month, x='yearMonth', y='count', color="site")
fig.show()

## Analyses sur les mots-clés

On crée une fonction qui nous renvoie les mots-clés et leurs nombres d'itérations pour un corpus donné.

In [9]:
def getKeywords(filesNames):
    keywords = {}
    for file in filesNames:
        f = open(file, 'r', encoding='utf-8')
        data = json.loads(f.read())
        f.close()
        years = data["data-all"].keys()
        for year in years:
            months = data["data-all"][year].keys()
            for month in months:
                days = data["data-all"][year][month].keys()
                for day in days:
                    articles = data["data-all"][year][month][day]
                    for article in articles:
                        kws = article["kws"]
                        for kw,iters in kws.items():
                            if(kw in keywords):
                                keywords[kw] += iters
                            else:
                                keywords[kw] = iters
    #Sort
    keywords = sorted(keywords.items(), key=lambda x:x[1])

    commonWordsDictionnaired = []
    for f,s in keywords:
        commonWordsDictionnaired.append({"keyword": f, "iteration": s})
    return commonWordsDictionnaired

In [10]:
commonWords = getKeywords(corpusFiles)[-50:]

On crée un bar chart des 100 mots-clés les plus communs du corpus.

In [11]:
fig = px.bar(commonWords, x='keyword', y='iteration')
fig.show()

### Analyses plus poussés sur les mots-clés

#### Voir les liens entre les mots-clés

On crée une fonction qui donne pour un corpus donné et un mot-clé donné les mots-clés relatifs au mot-clé choisi.

In [46]:
def showLinkedKws(corpus, pickedKw):
    linkedKws = {}
    for file in corpus:
        f = open(file, 'r', encoding='utf-8')
        data = json.loads(f.read())
        f.close()
        years = data["data-all"].keys()
        for year in years:
            months = data["data-all"][year].keys()
            for month in months:
                days = data["data-all"][year][month].keys()
                for day in days:
                    articles = data["data-all"][year][month][day]
                    for article in articles:
                        kws = article["kws"]
                        if(pickedKw in kws):
                            for kw,iters in kws.items():
                                if(kw != pickedKw):
                                    if(kw in linkedKws):
                                        linkedKws[kw] += 1
                                    else:
                                        linkedKws[kw] = 1
            #Sort
    linkedKws = sorted(linkedKws.items(), key=lambda x:x[1])
    commonLinkedWordsDictionnaired = []

    for f,s in linkedKws:
         commonLinkedWordsDictionnaired.append({"keyword": f, "iteration": s})
    return commonLinkedWordsDictionnaired

Un exemple avec le mot-clé "juif" ( mot totalement choisi au hasard :) )

In [48]:
pickedKeyword = "juif"

commonLinkedWords = showLinkedKws(corpusFiles, pickedKeyword)[-50:]

fig = px.bar(commonLinkedWords, x='keyword', y='iteration', title=pickedKeyword)
fig.show()

In [44]:
def showLinkedPeople(corpus, pickedKw):
    linkedPeople = {}
    for file in corpus:
        f = open(file, 'r', encoding='utf-8')
        data = json.loads(f.read())
        f.close()
        years = data["data-all"].keys()
        for year in years:
            months = data["data-all"][year].keys()
            for month in months:
                days = data["data-all"][year][month].keys()
                for day in days:
                    articles = data["data-all"][year][month][day]
                    for article in articles:
                        people = article["per"]
                        kws = article["kws"]

                        if(pickedKw in kws):
                            for p,iter in people.items():
                                if(p in linkedPeople):
                                    linkedPeople[p] += 1
                                else:
                                    linkedPeople[p] = 1
            #Sort
    linkedPeople = sorted(linkedPeople.items(), key=lambda x:x[1])
    commonLinkedPeople = []

    for f,s in linkedPeople:
        commonLinkedPeople.append({"person": f, "iteration": s})
    return commonLinkedPeople

In [45]:
pickedKeyword = "juif"

commonLinkedPeople = showLinkedPeople(corpusFiles, pickedKeyword)[-50:]

fig = px.bar(commonLinkedPeople, x='person', y='iteration',title=pickedKeyword)
fig.show()

Avec plusieurs keywords on a souvent le nom de "Alain Soral" qui revient.

On crée une nouvelle fonction qui nous donne aussi le site.

In [41]:
def showLinkedPeopleWithSite(corpus, pickedKw):
    linkedPeople = []
    for file in corpus:
        f = open(file, 'r', encoding='utf-8')
        data = json.loads(f.read())
        f.close()
        siteName = f.name.split(".")[1]
        print(siteName)
        years = data["data-all"].keys()
        for year in years:
            months = data["data-all"][year].keys()
            for month in months:
                days = data["data-all"][year][month].keys()
                for day in days:
                    articles = data["data-all"][year][month][day]
                    for article in articles:
                        people = article["per"]
                        kws = article["kws"]
                        if(pickedKw in kws):
                            for p,iter in people.items():
                                # Tester si la personne associé au site existe déjà
                                isNew = True
                                """for e in linkedPeople:
                                    if e["person"] == p and e["site"] == siteName:
                                        e["iteration"] += iter
                                        isNew = False
                                        break"""
                                if(isNew):
                                    linkedPeople.append({"person": p, "iteration": iter, "site": siteName})

     #BubbleSort
    good = False

    while(not good):
        good = True
        for i in range(0, len(linkedPeople) - 1):
                iter1 = linkedPeople[i]["iteration"]
                iter2 = linkedPeople[i+1]["iteration"]
                if(iter2 > iter1):
                    linkedPeople[i], linkedPeople[i+1] = linkedPeople[i+1], linkedPeople[i]
                    good = False

    return linkedPeople

In [43]:
pickedKeyword = "noir"

commonLinkedPeopleWithSite = showLinkedPeopleWithSite(corpusFiles, pickedKeyword)[0:50]

fig = px.bar(commonLinkedPeopleWithSite, x='person', y='iteration', color='site', title=pickedKeyword)
fig.show()

egaliteetreconciliation
sputniknews
presstv
fdesouche
